## Install pyTigerGraph

In [1]:
# Setup
!pip install pyTigerGraph

To-do: include db/ unique edges

## Add Imports and Establish Initial Connection

In [2]:
# Imports
import pyTigerGraph as tg
import json
import pandas as pd

# Connection parameters
hostName = "https://language.i.tgcloud.io"
userName = "tigergraph"
password = "tigergraph"

conn = tg.TigerGraphConnection(host=hostName, username=userName, password=password)

print("Connected")

Connected


## Clone the Data

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
# cd '/content/drive/MyDrive'

In [5]:
# !git clone https://github.com/sudha-vijayakumar/LanguageModel.git

In [6]:
root = '/Users/sudha.vijayakumar/Documents/GitHub/LanguageModel/Curated_Data/'

## Define and Publish the Schema

In [7]:
rel_type = pd.read_csv(root+'csv_imports/encoded/type_rel.csv')
rel = rel_type['type']

str_=""
for r in rel:
  str_+="CREATE DIRECTED EDGE "+r+"(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE=\"reverse_"+r+"\""
  str_+="\n"
print(str_)

CREATE DIRECTED EDGE Attribute(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_Attribute"
CREATE DIRECTED EDGE InSynset(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_InSynset"
CREATE DIRECTED EDGE Domain(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_Domain"
CREATE DIRECTED EDGE SimilarTo(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_SimilarTo"
CREATE DIRECTED EDGE AlsoSee(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_AlsoSee"
CREATE DIRECTED EDGE IsA(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_IsA"
CREATE DIRECTED EDGE PartOf(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM wo

In [8]:
# DEFINE / CREATE ALL EDGES AND VERTICES 
results = conn.gsql('''
  USE GLOBAL
  CREATE VERTEX synset(PRIMARY_ID id STRING, definition STRING, pos STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
  CREATE VERTEX word(PRIMARY_ID id STRING, name STRING, pos STRING, conceptUri STRING, label STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
  CREATE VERTEX part_of_speech(PRIMARY_ID id STRING, type STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
  CREATE DIRECTED EDGE has_pos(FROM synset, TO part_of_speech|FROM word, TO part_of_speech) WITH REVERSE_EDGE="reverse_has_pos"
  CREATE DIRECTED EDGE is_pos_of(FROM part_of_speech, TO synset|FROM part_of_speech, TO word) WITH REVERSE_EDGE="reverse_is_pos_of"

  CREATE DIRECTED EDGE Attribute(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_Attribute"
  CREATE DIRECTED EDGE InSynset(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_InSynset"
  CREATE DIRECTED EDGE Domain(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_Domain"
  CREATE DIRECTED EDGE SimilarTo(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_SimilarTo"
  CREATE DIRECTED EDGE AlsoSee(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_AlsoSee"
  CREATE DIRECTED EDGE IsA(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_IsA"
  CREATE DIRECTED EDGE PartOf(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_PartOf"
  CREATE DIRECTED EDGE Entailment(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_Entailment"
  CREATE DIRECTED EDGE VerbGroup(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_VerbGroup"
  CREATE DIRECTED EDGE Cause(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_Cause"
  CREATE DIRECTED EDGE Antonym(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_Antonym"
  CREATE DIRECTED EDGE AtLocation(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_AtLocation"
  CREATE DIRECTED EDGE CapableOf(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_CapableOf"
  CREATE DIRECTED EDGE Causes(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_Causes"
  CREATE DIRECTED EDGE CausesDesire(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_CausesDesire"
  CREATE DIRECTED EDGE CreatedBy(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_CreatedBy"
  CREATE DIRECTED EDGE DefinedAs(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_DefinedAs"
  CREATE DIRECTED EDGE DerivedFrom(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_DerivedFrom"
  CREATE DIRECTED EDGE Desires(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_Desires"
  CREATE DIRECTED EDGE DistinctFrom(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_DistinctFrom"
  CREATE DIRECTED EDGE Entails(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_Entails"
  CREATE DIRECTED EDGE EtymologicallyDerivedFrom(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_EtymologicallyDerivedFrom"
  CREATE DIRECTED EDGE EtymologicallyRelatedTo(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_EtymologicallyRelatedTo"
  CREATE DIRECTED EDGE FormOf(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_FormOf"
  CREATE DIRECTED EDGE HasA(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_HasA"
  CREATE DIRECTED EDGE HasContext(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_HasContext"
  CREATE DIRECTED EDGE HasFirstSubevent(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_HasFirstSubevent"
  CREATE DIRECTED EDGE HasLastSubevent(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_HasLastSubevent"
  CREATE DIRECTED EDGE HasPrerequisite(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_HasPrerequisite"
  CREATE DIRECTED EDGE HasProperty(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_HasProperty"
  CREATE DIRECTED EDGE HasSubevent(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_HasSubevent"
  CREATE DIRECTED EDGE InstanceOf(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_InstanceOf"
  CREATE DIRECTED EDGE LocatedNear(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_LocatedNear"
  CREATE DIRECTED EDGE MadeOf(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_MadeOf"
  CREATE DIRECTED EDGE MannerOf(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_MannerOf"
  CREATE DIRECTED EDGE MotivatedByGoal(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_MotivatedByGoal"
  CREATE DIRECTED EDGE NotCapableOf(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_NotCapableOf"
  CREATE DIRECTED EDGE NotDesires(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_NotDesires"
  CREATE DIRECTED EDGE NotHasProperty(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_NotHasProperty"
  CREATE DIRECTED EDGE ReceivesAction(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_ReceivesAction"
  CREATE DIRECTED EDGE RelatedTo(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_RelatedTo"
  CREATE DIRECTED EDGE SymbolOf(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_SymbolOf"
  CREATE DIRECTED EDGE Synonym(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_Synonym"
  CREATE DIRECTED EDGE UsedFor(FROM synset, TO synset|FROM synset, TO word|FROM word, TO synset|FROM word, TO word) WITH REVERSE_EDGE="reverse_UsedFor"
  ''')
print(results)

Successfully created vertex types: [synset].
Successfully created vertex types: [word].
Successfully created vertex types: [part_of_speech].
Successfully created edge types: [has_pos].
Successfully created reverse edge types: [reverse_has_pos].
Successfully created edge types: [is_pos_of].
Successfully created reverse edge types: [reverse_is_pos_of].
Successfully created edge types: [Attribute].
Successfully created reverse edge types: [reverse_Attribute].
Successfully created edge types: [InSynset].
Successfully created reverse edge types: [reverse_InSynset].
Successfully created edge types: [Domain].
Successfully created reverse edge types: [reverse_Domain].
Successfully created edge types: [SimilarTo].
Successfully created reverse edge types: [reverse_SimilarTo].
Successfully created edge types: [AlsoSee].
Successfully created reverse edge types: [reverse_AlsoSee].
Successfully created edge types: [IsA].
Successfully created reverse edge types: [reverse_IsA].
Successfully created ed

# WordNET

## Create WordNet Graph

In [9]:
rel = rel_type['type']

str_=""
for r in rel:
  str_+=r+","
print(str_)

Attribute,InSynset,Domain,SimilarTo,AlsoSee,IsA,PartOf,Entailment,VerbGroup,Cause,Antonym,AtLocation,CapableOf,Causes,CausesDesire,CreatedBy,DefinedAs,DerivedFrom,Desires,DistinctFrom,Entails,EtymologicallyDerivedFrom,EtymologicallyRelatedTo,FormOf,HasA,HasContext,HasFirstSubevent,HasLastSubevent,HasPrerequisite,HasProperty,HasSubevent,InstanceOf,LocatedNear,MadeOf,MannerOf,MotivatedByGoal,NotCapableOf,NotDesires,NotHasProperty,ReceivesAction,RelatedTo,SymbolOf,Synonym,UsedFor,dbpedia/capital,dbpedia/field,dbpedia/genre,dbpedia/genus,dbpedia/influencedBy,dbpedia/knownFor,dbpedia/language,dbpedia/leader,dbpedia/occupation,dbpedia/product,


In [10]:
results = conn.gsql('CREATE GRAPH ConceptNet(synset, word, part_of_speech, Attribute,InSynset,Domain,SimilarTo,AlsoSee,IsA,PartOf,Entailment,VerbGroup,Cause,Antonym,AtLocation,CapableOf,Causes,CausesDesire,CreatedBy,DefinedAs,DerivedFrom,Desires,DistinctFrom,Entails,EtymologicallyDerivedFrom,EtymologicallyRelatedTo,FormOf,HasA,HasContext,HasFirstSubevent,HasLastSubevent,HasPrerequisite,HasProperty,HasSubevent,InstanceOf,LocatedNear,MadeOf,MannerOf,MotivatedByGoal,NotCapableOf,NotDesires,NotHasProperty,ReceivesAction,RelatedTo,SymbolOf,Synonym,UsedFor,has_pos, is_pos_of)')
print(results)

The graph ConceptNet is created.


In [11]:
conn.graphname="ConceptNet"
secret = conn.createSecret()
print(secret)
authToken = conn.getToken(secret)
authToken = authToken[0]
print(authToken)
# authToken = 'rc7reopbis1667ksgcppq5v5fb99p6s1'
conn = tg.TigerGraphConnection(host=hostName, graphname="ConceptNet", username=userName, password=password, apiToken=authToken)

def pprint(string):
  print(json.dumps(string, indent=2))

omlqveaiq0gmtmpiuvn6mppqc1kb2u23
kgprarmhs1cj04s0qa8ibo292a731r9l


## Create Loading Jobs

### Word

Let's take a look at what one of our files looks like so we can write a loading job.

In [12]:
!head -n 2 root+'csv_imports/WN-nodes.csv'

head: root+csv_imports/WN-nodes.csv: No such file or directory


Here it's important to note that the `$0`, `$1` values line up with the columns of your data.
In this example:
- `$0` is the `uri` column,
- `$1` is `id`,
- `$2` is `word`
- and so on

In [13]:
rel_type = pd.read_csv(root+'csv_imports/encoded/type_rel.csv')
rel = rel_type['type']

str_=""
for r in rel:
  str_+="LOAD MyDataSource TO EDGE  "+r+" VALUES($0 synset, $1 synset) WHERE $4 == \""+r+"\" USING SEPARATOR=\",\""", HEADER=\"true\""""
  str_+=";\n"
print(str_)

LOAD MyDataSource TO EDGE  Attribute VALUES($0 synset, $1 synset) WHERE $4 == "Attribute" USING SEPARATOR=",", HEADER="true";
LOAD MyDataSource TO EDGE  InSynset VALUES($0 synset, $1 synset) WHERE $4 == "InSynset" USING SEPARATOR=",", HEADER="true";
LOAD MyDataSource TO EDGE  Domain VALUES($0 synset, $1 synset) WHERE $4 == "Domain" USING SEPARATOR=",", HEADER="true";
LOAD MyDataSource TO EDGE  SimilarTo VALUES($0 synset, $1 synset) WHERE $4 == "SimilarTo" USING SEPARATOR=",", HEADER="true";
LOAD MyDataSource TO EDGE  AlsoSee VALUES($0 synset, $1 synset) WHERE $4 == "AlsoSee" USING SEPARATOR=",", HEADER="true";
LOAD MyDataSource TO EDGE  IsA VALUES($0 synset, $1 synset) WHERE $4 == "IsA" USING SEPARATOR=",", HEADER="true";
LOAD MyDataSource TO EDGE  PartOf VALUES($0 synset, $1 synset) WHERE $4 == "PartOf" USING SEPARATOR=",", HEADER="true";
LOAD MyDataSource TO EDGE  Entailment VALUES($0 synset, $1 synset) WHERE $4 == "Entailment" USING SEPARATOR=",", HEADER="true";
LOAD MyDataSource TO

In [14]:
results = conn.gsql('''
  USE GRAPH ConceptNet
  BEGIN
  CREATE LOADING JOB load_job_relationships FOR GRAPH ConceptNet {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO EDGE  Attribute VALUES($0 synset, $1 synset) WHERE $4 == "Attribute" USING SEPARATOR=",", HEADER="true";
      LOAD MyDataSource TO EDGE  InSynset VALUES($0 synset, $1 synset) WHERE $4 == "InSynset" USING SEPARATOR=",", HEADER="true";
      LOAD MyDataSource TO EDGE  Domain VALUES($0 synset, $1 synset) WHERE $4 == "Domain" USING SEPARATOR=",", HEADER="true";
      LOAD MyDataSource TO EDGE  SimilarTo VALUES($0 synset, $1 synset) WHERE $4 == "SimilarTo" USING SEPARATOR=",", HEADER="true";
      LOAD MyDataSource TO EDGE  AlsoSee VALUES($0 synset, $1 synset) WHERE $4 == "AlsoSee" USING SEPARATOR=",", HEADER="true";
      LOAD MyDataSource TO EDGE  IsA VALUES($0 synset, $1 synset) WHERE $4 == "IsA" USING SEPARATOR=",", HEADER="true";
      LOAD MyDataSource TO EDGE  PartOf VALUES($0 synset, $1 synset) WHERE $4 == "PartOf" USING SEPARATOR=",", HEADER="true";
      LOAD MyDataSource TO EDGE  Entailment VALUES($0 synset, $1 synset) WHERE $4 == "Entailment" USING SEPARATOR=",", HEADER="true";
      LOAD MyDataSource TO EDGE  VerbGroup VALUES($0 synset, $1 synset) WHERE $4 == "VerbGroup" USING SEPARATOR=",", HEADER="true";
      LOAD MyDataSource TO EDGE  Cause VALUES($0 synset, $1 synset) WHERE $4 == "Cause" USING SEPARATOR=",", HEADER="true";
      LOAD MyDataSource TO EDGE  Antonym VALUES($0 synset, $1 synset) WHERE $4 == "Antonym" USING SEPARATOR=",", HEADER="true";
      LOAD MyDataSource TO EDGE  AtLocation VALUES($0 synset, $1 synset) WHERE $4 == "AtLocation" USING SEPARATOR=",", HEADER="true";
      LOAD MyDataSource TO EDGE  CapableOf VALUES($0 synset, $1 synset) WHERE $4 == "CapableOf" USING SEPARATOR=",", HEADER="true";
      LOAD MyDataSource TO EDGE  Causes VALUES($0 synset, $1 synset) WHERE $4 == "Causes" USING SEPARATOR=",", HEADER="true";
      LOAD MyDataSource TO EDGE  CausesDesire VALUES($0 synset, $1 synset) WHERE $4 == "CausesDesire" USING SEPARATOR=",", HEADER="true";
      LOAD MyDataSource TO EDGE  CreatedBy VALUES($0 synset, $1 synset) WHERE $4 == "CreatedBy" USING SEPARATOR=",", HEADER="true";
      LOAD MyDataSource TO EDGE  DefinedAs VALUES($0 synset, $1 synset) WHERE $4 == "DefinedAs" USING SEPARATOR=",", HEADER="true";
      LOAD MyDataSource TO EDGE  DerivedFrom VALUES($0 synset, $1 synset) WHERE $4 == "DerivedFrom" USING SEPARATOR=",", HEADER="true";
      LOAD MyDataSource TO EDGE  Desires VALUES($0 synset, $1 synset) WHERE $4 == "Desires" USING SEPARATOR=",", HEADER="true";
      LOAD MyDataSource TO EDGE  DistinctFrom VALUES($0 synset, $1 synset) WHERE $4 == "DistinctFrom" USING SEPARATOR=",", HEADER="true";
      LOAD MyDataSource TO EDGE  Entails VALUES($0 synset, $1 synset) WHERE $4 == "Entails" USING SEPARATOR=",", HEADER="true";
      LOAD MyDataSource TO EDGE  EtymologicallyDerivedFrom VALUES($0 synset, $1 synset) WHERE $4 == "EtymologicallyDerivedFrom" USING SEPARATOR=",", HEADER="true";
      LOAD MyDataSource TO EDGE  EtymologicallyRelatedTo VALUES($0 synset, $1 synset) WHERE $4 == "EtymologicallyRelatedTo" USING SEPARATOR=",", HEADER="true";
      LOAD MyDataSource TO EDGE  FormOf VALUES($0 synset, $1 synset) WHERE $4 == "FormOf" USING SEPARATOR=",", HEADER="true";
      LOAD MyDataSource TO EDGE  HasA VALUES($0 synset, $1 synset) WHERE $4 == "HasA" USING SEPARATOR=",", HEADER="true";
      LOAD MyDataSource TO EDGE  HasContext VALUES($0 synset, $1 synset) WHERE $4 == "HasContext" USING SEPARATOR=",", HEADER="true";
      LOAD MyDataSource TO EDGE  HasFirstSubevent VALUES($0 synset, $1 synset) WHERE $4 == "HasFirstSubevent" USING SEPARATOR=",", HEADER="true";
      LOAD MyDataSource TO EDGE  HasLastSubevent VALUES($0 synset, $1 synset) WHERE $4 == "HasLastSubevent" USING SEPARATOR=",", HEADER="true";
      LOAD MyDataSource TO EDGE  HasPrerequisite VALUES($0 synset, $1 synset) WHERE $4 == "HasPrerequisite" USING SEPARATOR=",", HEADER="true";
      LOAD MyDataSource TO EDGE  HasProperty VALUES($0 synset, $1 synset) WHERE $4 == "HasProperty" USING SEPARATOR=",", HEADER="true";
      LOAD MyDataSource TO EDGE  HasSubevent VALUES($0 synset, $1 synset) WHERE $4 == "HasSubevent" USING SEPARATOR=",", HEADER="true";
      LOAD MyDataSource TO EDGE  InstanceOf VALUES($0 synset, $1 synset) WHERE $4 == "InstanceOf" USING SEPARATOR=",", HEADER="true";
      LOAD MyDataSource TO EDGE  LocatedNear VALUES($0 synset, $1 synset) WHERE $4 == "LocatedNear" USING SEPARATOR=",", HEADER="true";
      LOAD MyDataSource TO EDGE  MadeOf VALUES($0 synset, $1 synset) WHERE $4 == "MadeOf" USING SEPARATOR=",", HEADER="true";
      LOAD MyDataSource TO EDGE  MannerOf VALUES($0 synset, $1 synset) WHERE $4 == "MannerOf" USING SEPARATOR=",", HEADER="true";
      LOAD MyDataSource TO EDGE  MotivatedByGoal VALUES($0 synset, $1 synset) WHERE $4 == "MotivatedByGoal" USING SEPARATOR=",", HEADER="true";
      LOAD MyDataSource TO EDGE  NotCapableOf VALUES($0 synset, $1 synset) WHERE $4 == "NotCapableOf" USING SEPARATOR=",", HEADER="true";
      LOAD MyDataSource TO EDGE  NotDesires VALUES($0 synset, $1 synset) WHERE $4 == "NotDesires" USING SEPARATOR=",", HEADER="true";
      LOAD MyDataSource TO EDGE  NotHasProperty VALUES($0 synset, $1 synset) WHERE $4 == "NotHasProperty" USING SEPARATOR=",", HEADER="true";
      LOAD MyDataSource TO EDGE  ReceivesAction VALUES($0 synset, $1 synset) WHERE $4 == "ReceivesAction" USING SEPARATOR=",", HEADER="true";
      LOAD MyDataSource TO EDGE  RelatedTo VALUES($0 synset, $1 synset) WHERE $4 == "RelatedTo" USING SEPARATOR=",", HEADER="true";
      LOAD MyDataSource TO EDGE  SymbolOf VALUES($0 synset, $1 synset) WHERE $4 == "SymbolOf" USING SEPARATOR=",", HEADER="true";
      LOAD MyDataSource TO EDGE  Synonym VALUES($0 synset, $1 synset) WHERE $4 == "Synonym" USING SEPARATOR=",", HEADER="true";
      LOAD MyDataSource TO EDGE  UsedFor VALUES($0 synset, $1 synset) WHERE $4 == "UsedFor" USING SEPARATOR=",", HEADER="true";

    }

    
    CREATE LOADING JOB load_job_pos_wn FOR GRAPH ConceptNet {
          DEFINE FILENAME MyDataSource;
          LOAD MyDataSource TO VERTEX part_of_speech VALUES($2, $1) USING SEPARATOR=",", HEADER="true", EOL="\n";
        }

  
    CREATE LOADING JOB load_job_synsets_encoded FOR GRAPH ConceptNet {
          DEFINE FILENAME MyDataSource;
          LOAD MyDataSource TO VERTEX synset VALUES($0, $2, $1) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
        }

    
    CREATE LOADING JOB load_job_words_encoded FOR GRAPH ConceptNet {
          DEFINE FILENAME MyDataSource;
          LOAD MyDataSource TO VERTEX word VALUES($0, $1, $2, $3, $4) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
          LOAD MyDataSource TO EDGE has_pos VALUES($0 word, $2 part_of_speech) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
          LOAD MyDataSource TO EDGE is_pos_of VALUES($2 part_of_speech, $0 word) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
          LOAD MyDataSource TO EDGE has_pos VALUES($0 synset, $2 part_of_speech) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
          LOAD MyDataSource TO EDGE is_pos_of VALUES($2 part_of_speech, $0 synset) USING SEPARATOR=",", HEADER="true", EOL="\n", QUOTE="double";
        }



  END
  ''')
print(results)

Using graph 'ConceptNet'
Successfully created loading jobs: [load_job_relationships].
Successfully created loading jobs: [load_job_pos_wn].
Successfully created loading jobs: [load_job_synsets_encoded].
Successfully created loading jobs: [load_job_words_encoded].


## Load Data

### Words

In [15]:
# Load the posts file wiht the 'load_words' job
load_words = root+'csv_imports/relationships.csv'
results = conn.uploadFile(load_words, timeout=200000,fileTag='MyDataSource', jobName='load_job_relationships')
# print(json.dumps(results, indent=2))

In [16]:
# Load the posts file wiht the 'load_words' job
load_words = root+'csv_imports/encoded/pos_wn.csv'
results = conn.uploadFile(load_words, timeout=100000,fileTag='MyDataSource', jobName='load_job_pos_wn')
print(json.dumps(results, indent=2))

[
  {
    "sourceFileName": "Online_POST",
    "statistics": {
      "validLine": 6,
      "rejectLine": 0,
      "failedConditionLine": 0,
      "notEnoughToken": 0,
      "invalidJson": 0,
      "oversizeToken": 0,
      "vertex": [
        {
          "typeName": "part_of_speech",
          "validObject": 6,
          "noIdFound": 0,
          "invalidAttribute": 0,
          "invalidVertexType": 0,
          "invalidPrimaryId": 0,
          "invalidSecondaryId": 0,
          "incorrectFixedBinaryLength": 0
        }
      ],
      "edge": [],
      "deleteVertex": [],
      "deleteEdge": []
    }
  }
]


### Edges

In [17]:
# Load the posts file wiht the 'load_edges' job
load_edges = root+'/csv_imports/synsets.csv'
results = conn.uploadFile(load_edges, timeout=100000, fileTag='MyDataSource', jobName='load_job_synsets_encoded')
print(json.dumps(results, indent=2))

[
  {
    "sourceFileName": "Online_POST",
    "statistics": {
      "validLine": 117660,
      "rejectLine": 0,
      "failedConditionLine": 0,
      "notEnoughToken": 0,
      "invalidJson": 0,
      "oversizeToken": 0,
      "vertex": [
        {
          "typeName": "synset",
          "validObject": 117660,
          "noIdFound": 0,
          "invalidAttribute": 0,
          "invalidVertexType": 0,
          "invalidPrimaryId": 0,
          "invalidSecondaryId": 0,
          "incorrectFixedBinaryLength": 0
        }
      ],
      "edge": [],
      "deleteVertex": [],
      "deleteEdge": []
    }
  }
]


In [ ]:
# Load the posts file wiht the 'load_edges' job
load_edges = root+'csv_imports/encoded/words_encoded.csv'
results = conn.uploadFile(load_edges, timeout=100000, fileTag='MyDataSource', jobName='load_job_words_encoded')
print(json.dumps(results, indent=2))

## Exploring the Graph

### Get Vertex and Edge Schema

In [ ]:
results = conn.getVertexTypes()
print(f"Verticies: {results}")
vertices = results

results = conn.getEdgeTypes()
print(f"Edges: {results}")
edges = results

In [ ]:

print(f"Results for Post vertex")
pprint(conn.getVertexType("word"))

print("-----------------")
print(f"Results for liked edge")
pprint(conn.getEdgeType("type"))


## Counting Data

In [ ]:
print("Vertex Counts")
for vertex in vertices:
  print(f"There are {conn.getVertexCount(vertex)} {vertex} vertices in the graph")

print("--------------")
print("Edge Counts")
for edge in edges:
  print(f"There are {conn.getEdgeCount(edge)} {edge} edges in the graph")